# Event viewer

calin/examples/iact_data/event viewer.ipynb - Stephen Fegan - 2016-12-13

Copyright 2016, Stephen Fegan <sfegan@llr.in2p3.fr>
Laboratoire Leprince-Ringuet, CNRS/IN2P3, Ecole Polytechnique, Institut Polytechnique de Paris

This file is part of "__calin__". "__calin__" is free software: you can redistribute it and/or modify it under the
terms of the GNU General Public License version 2 or later, as published by
the Free Software Foundation. "__calin__" is distributed in the hope that it will be useful, but WITHOUT ANY WARRANTY; without even the implied warranty of MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the GNU General Public License for more details.

## Introduction

Open a ZFits file and draw camera events

In [ ]:
%pylab inline
import calin.iact_data.telescope_data_source
from IPython.display import clear_output
from ipywidgets.widgets import *

In [ ]:
dcfg = calin.iact_data.telescope_data_source.NectarCamZFITSDataSource.default_decoder_config()
dcfg.set_demand_configured_module_id([1,2,5,6,10,11,14,15,17,18])
dcfg.set_exchange_gain_channels(True)
dcfg.set_camera_type(dcfg.NECTARCAM_TESTBENCH_19CHANNEL)
src = calin.iact_data.telescope_data_source.NectarCamZFITSDataSource(
    '/CTA/cta.cppm.in2p3.fr/NectarCAM/20161021/Run0264.fits.fz', dcfg)

In [ ]:
run_config = src.get_run_configuration()

In [ ]:
def plot_image(pix_data):
#    figure(figsize=(10,8))
#    figure(figsize=(7,6))
    figure(figsize=(5,4.5))
    pix = []
    max_xy = 0
    for pix_index in range(len(pix_data)):
        pix_id = run_config.configured_channel_id(pix_index)
        vx = run_config.camera_layout().channel(pix_id).outline_polygon_vertex_x_view()
        vy = run_config.camera_layout().channel(pix_id).outline_polygon_vertex_y_view()
        vv = zeros((len(vx),2))
        vv[:,0] = vx
        vv[:,1] = vy
        max_xy = max(max_xy, max(abs(vx)), max(abs(vy)))
        pix.append(Polygon(vv,closed=True))
    pc = matplotlib.collections.PatchCollection(pix, cmap=matplotlib.cm.jet)
    pc.set_array(asarray(pix_data))
    pc.set_linewidths(0)
    clo = min(-100, min(pix_data))
    chi = max(100, max(pix_data))
    pc.set_clim(clo,chi)
    gca().add_collection(pc)
    axis('square')
    axis(asarray([-1,1,-1,1])*1.05*max_xy)
    xlabel('X coordinate [cm]')
    ylabel('Y coordinate [cm]')
    colorbar(pc)
#    grid(color='w')

In [ ]:
def plot_next_event():
    index, event, arena = src.get_next()
    im = event.high_gain_image()
    pixdata = im.camera_charges().charge() * 0.0
    for i in range(im.camera_waveforms().waveform_size()):
        samp = im.camera_waveforms().waveform(i).samples()
        pixdata[i] = sum(samp[30:42]*1.0) - sum(samp[0:12]*1.0)
    plot_image(pixdata)
    title('Index: %d'%index)

In [ ]:
button  = Button(description="Display next event")
display(button)

def on_button_clicked(b):
    clear_output()
    plot_next_event()

button.on_click(on_button_clicked)